In [1]:
import numpy as np
import pandas as pd
from apyori import apriori

In [2]:
# df = pd.read_csv('../association_rule_data_01.csv', header = None)
df = pd.read_csv('../association_rule_data_01.csv')
df.head()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
ds = df.values

In [4]:
tr = []

for i in range(ds.shape[0]):
    tr.append([str(ds[i][j]) for j in range(ds.shape[1])])

In [5]:
tr[1]

['chutney',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan']

In [6]:
rules = apriori(tr, min_support = 0.003, min_confidence = 0.2, 
                min_lift = 5, min_length = 2)
results = list(rules)

In [7]:
df.values[:2, :]

array([['burgers', 'meatballs', 'eggs', nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['chutney', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan]], dtype=object)

In [8]:
rules = apriori(df.values, min_support = 0.003, min_confidence = 0.2, 
                min_lift = 5, min_length = 2)
# results = list(rules)

In [9]:
pd.DataFrame([results[0]])

,items,support,ordered_statistics
0,"(fromage blanc, honey)",0.003333,"[((fromage blanc), (honey), 0.2450980392156863..."


In [10]:
df_rule = pd.DataFrame(results)
df_rule.head()

,items,support,ordered_statistics
0,"(fromage blanc, honey)",0.003333,"[((fromage blanc), (honey), 0.2450980392156863..."
1,"(nan, fromage blanc, honey)",0.003333,"[((fromage blanc), (nan, honey), 0.24509803921..."
2,"(spaghetti, ground beef, tomato sauce)",0.003067,"[((tomato sauce), (spaghetti, ground beef), 0...."
3,"(olive oil, whole wheat pasta, mineral water)",0.003867,"[((whole wheat pasta, mineral water), (olive o..."
4,"(olive oil, milk, frozen vegetables, mineral w...",0.003333,"[((olive oil, frozen vegetables), (milk, miner..."


In [11]:
colnames = pd.DataFrame(np.array(results[0])[2]).columns.values.tolist()
colnames

['items_base', 'items_add', 'confidence', 'lift']

In [12]:
df_rule_stat = pd.DataFrame(df_rule.iloc[:, 2].apply(lambda x: x[0])).explode("ordered_statistics")
df_rule_stat["cols"] = colnames * (df_rule_stat.index.values.max() + 1)
df_rule_stat = df_rule_stat.reset_index()
df_rule_stat.head()

,index,ordered_statistics,cols
0,0,(fromage blanc),items_base
1,0,(honey),items_add
2,0,0.245098,confidence
3,0,5.17813,lift
4,1,(fromage blanc),items_base


In [13]:
df_rule_stat_pivot = df_rule_stat.pivot(index="index", columns="cols", values="ordered_statistics").reset_index(drop = True)
df_rule_stat_pivot

cols,confidence,items_add,items_base,lift
0,0.245098,(honey),(fromage blanc),5.17813
1,0.245098,"(nan, honey)",(fromage blanc),5.17813
2,0.216981,"(spaghetti, ground beef)",(tomato sauce),5.53523
3,0.402778,(olive oil),"(whole wheat pasta, mineral water)",6.12745
4,0.294118,"(milk, mineral water)","(olive oil, frozen vegetables)",6.12745
5,0.383333,"(milk, mineral water)","(frozen vegetables, soup)",7.98611
6,0.216981,"(spaghetti, nan, ground beef)",(tomato sauce),5.53523
7,0.402778,"(olive oil, nan)","(whole wheat pasta, mineral water)",6.12745
8,0.294118,"(milk, nan, mineral water)","(olive oil, frozen vegetables)",6.12745
9,0.383333,"(milk, nan, mineral water)","(frozen vegetables, soup)",7.98611


In [16]:
df_rule_stat_pivot["support"] = df_rule["support"]
df_rule_stat_pivot.head()

cols,confidence,items_add,items_base,lift,support
0,0.245098,(honey),(fromage blanc),5.17813,0.003333
1,0.245098,"(nan, honey)",(fromage blanc),5.17813,0.003333
2,0.216981,"(spaghetti, ground beef)",(tomato sauce),5.53523,0.003067
3,0.402778,(olive oil),"(whole wheat pasta, mineral water)",6.12745,0.003867
4,0.294118,"(milk, mineral water)","(olive oil, frozen vegetables)",6.12745,0.003333


In [17]:
df_rule_stat_pivot = df_rule_stat_pivot.iloc[:, [4, 0, 3, 2, 1]]
df_rule_stat_pivot.head()

cols,support,confidence,lift,items_base,items_add
0,0.003333,0.245098,5.17813,(fromage blanc),(honey)
1,0.003333,0.245098,5.17813,(fromage blanc),"(nan, honey)"
2,0.003067,0.216981,5.53523,(tomato sauce),"(spaghetti, ground beef)"
3,0.003867,0.402778,6.12745,"(whole wheat pasta, mineral water)",(olive oil)
4,0.003333,0.294118,6.12745,"(olive oil, frozen vegetables)","(milk, mineral water)"
